To work with the functions, I have assumed that the files for each target are in a folder with that target's name. This folder contains all the raw spectra, as well as a text file that lists the mode of each observation (HE or HR) named MODES.txt - an example of this is include in the repository, and also an example after analysis is included.
The final cell contains a 'pipeline' function that combines all other functions.

The cell below contains all necessary imports. Make sure that the variable ispec_dir has the correct path to access the iSpec files. If the message "Not optimized version loaded!" appears below this shouldn't be a problem. For any other error messages, ensuring all packages are fully update normally solves them; if this fails then try reinstalling iSpec.

In [1]:
import os
import shutil
from astropy.io import fits
import sys
import numpy as np
import logging
import multiprocessing
from multiprocessing import Pool
import csv
import pandas as pd
import matplotlib.pyplot as plt
import math 
from statistics import mean
ispec_dir = 'iSpec/' #Change this if necessary
sys.path.insert(0, os.path.abspath(ispec_dir))
import ispec
#--- Change LOG level ----------------------------------------------------------
#LOG_LEVEL = "warning"
LOG_LEVEL = "info"
logger = logging.getLogger() # root logger, common for all
logger.setLevel(logging.getLevelName(LOG_LEVEL.upper()))

*********************************************************************
Not optimized version loaded!
*********************************************************************
*********************************************************************
Not optimized version loaded!
*********************************************************************
*********************************************************************
Not optimized version loaded!
*********************************************************************


In [ ]:
Add the path to your files below.

In [ ]:
p = '/home/alix/Sophie data/SOPHIE_sd1_data/sd1_data/'

The separate_modes function reads the observation modes from the MODES file, and uses this to move the individual raw spectra to separate HE and HR directories within the target folder.

In [ ]:
def separate_modes(star_name):
    path = p +star_name
    
    raw_spectra = [i for i in os.listdir(path) if i.endswith('.fits')]
    file = open(path +'/MODES.txt', 'r')
    modes = file.readlines()
    mode = [i.rstrip('\n') for i in modes]
    if mode.count('HE') ==0:
        os.mkdir(path + '/HR')
        for i in raw_spectra:
            shutil.move(path + '/' +i, path + '/HR/' + i)
    elif mode.count('HR') ==0:
        os.mkdir(path + '/HE')
        for i in raw_spectra:
            shutil.move(path + '/' +i, path + '/HE/' + i)
    else:
        os.mkdir(path+'/HR')
        os.mkdir(path+'/HE')
        
        
        for i in range(len(mode)):
            print(mode[i])
            if mode[i] == 'HE':
                shutil.move(path + '/' +raw_spectra[i], path + '/HE/' + raw_spectra[i])
            elif mode[i] == 'HR':
                shutil.move(path + '/' +raw_spectra[i], path + '/HR/' + raw_spectra[i])
            

The HE and HR mode individual spectra must be coadded separately as they have different resolutions. The function below filters out any spectra with SNR<20.
The radial velocity correction is determined using the NARVAL solar template. 
In later functions, the line list GAIA v6 is used - this is optimised for observations with R=47000 so ideally we want the resolution of the coadded spectrum to match this. For the HR mode, the SOPHIE spectra can be degraded from R=75000 (make sure to change this number if using data from a different instrument), however HE mode spectra are too low resolution for this (check this if using a different instrument though).
A continuum is fit to the data using the splines method.
The spectra are then coadded by taking the average flux and flux error at 0.001nm wavelength steps, and the resulting spectrum is saved to the respective HE or HR directory.

In [1]:
def coadd_spectra(wavelength_min, wavelength_max, star_name, mode='HR'):
    if mode == 'HR':
        path = p +star_name +'/HR'
    if mode == 'HE':
        path = p +star_name +'/HE'
    raw_spectra = [i for i in os.listdir(path) if i.endswith('.fits')]
    spectra=[]
    snr_list=[]
    for i in range(len(raw_spectra)):
        spectrum = ispec.read_spectrum(path+'/'+raw_spectra[i])
        hdul = fits.open(path+'/'+raw_spectra[i])
        hdr=hdul[0].header
        snr = ispec.estimate_snr(spectrum['flux'], num_points=20)                              
        spectrum['err'] = spectrum['flux']/snr
        if snr> 20:
            snr_list.append(snr)
            logging.info("Radial velocity determination with template...")
            # - Telluric correction - uncomment if not already done in data reduction
            '''
            telluric_linelist_file = ispec_dir + "/input/linelists/CCF/Synth.Tellurics.500_1100nm/mask.lst"
            telluric_linelist = ispec.read_telluric_linelist(telluric_linelist_file, minimum_depth=0.0)

            models, ccf = ispec.cross_correlate_with_mask(spectrum, telluric_linelist, \
                                    lower_velocity_limit=-500, upper_velocity_limit=500, \
                                    velocity_step=0.5, mask_depth=0.01, \
                                    fourier = False,
                                    only_one_peak = True)

            vel_telluric = np.round(models[0].mu(), 2) # km/s
            vel_telluric_err = np.round(models[0].emu(), 2) # km/s
    
            tel_cor_spectrum = ispec.correct_velocity(spectrum, vel_telluric)
            
            '''
            template = ispec.read_spectrum(ispec_dir + "/input/spectra/templates/NARVAL.Sun.370_1048nm/template.txt.gz")
            models, ccf = ispec.cross_correlate_with_template(spectrum, template, \
                                    lower_velocity_limit=-200, upper_velocity_limit=200, \
                                    velocity_step=1.0, fourier=False)
            # Number of models represent the number of components
            components = len(models)
            # First component:
            rv = np.round(models[0].mu(), 2) # km/s
            rv_err = np.round(models[0].emu(), 2) # km/s
            cor_spectrum = ispec.correct_velocity(spectrum, rv)
            if mode == 'HR':
                from_resolution=75000 #Change for different spectrographs
                to_resolution = 47000
                deg_spectrum = ispec.convolve_spectrum(cor_spectrum, to_resolution, \
                                                        from_resolution=from_resolution)
            elif mode=='HE':
                deg_spectrum = cor_spectrum
             #--- Continuum fit -------------------------------------------------------------
            model = "Splines" # "Polynomy"
            degree = 2
            nknots = None # Automatic: 1 spline every 5 nm
            from_resolution = to_resolution = 47000

            # Strategy: Filter first median values and secondly MAXIMUMs in order to find the continuum
            order='median+max'
            median_wave_range=0.05
            max_wave_range=1.0

            star_continuum_model = ispec.fit_continuum(deg_spectrum, from_resolution=from_resolution, \
                                        nknots=nknots, degree=degree, \
                                        median_wave_range=median_wave_range, \
                                        max_wave_range=max_wave_range, \
                                        model=model, order=order, \
                                        automatic_strong_line_detection=True, \
                                        strong_line_probability=0.5, \
                                        use_errors_for_fitting=True)
            #--- Normalize -------------------------------------------------------------
            normalized_star_spectrum = ispec.normalize_spectrum(deg_spectrum, star_continuum_model, consider_continuum_errors=False)
            spectra.append(normalized_star_spectrum)
    wavelengths = np.arange(wavelength_min, wavelength_max, 0.001)
    flux=[]
    err=[]
    for i in spectra:
        if len(spectra)>=1:
            resampled_star_spectrum = ispec.resample_spectrum(i, wavelengths, method='linear', zero_edges=True)
    
            flux.append(resampled_star_spectrum['flux'])
            err.append(resampled_star_spectrum['err']) 
    if len(spectra)>=1:
        coadded_spectrum = ispec.create_spectrum_structure(wavelengths, flux = np.average(flux, axis=0),err=np.average(err, axis=0))
        spec_snr = ispec.estimate_snr(coadded_spectrum['flux'], num_points=20)
        smoothed_star_spectrum = ispec.convolve_spectrum(coadded_spectrum, 40000)
        snr_list.append(spec_snr)
        file=open(path +'/SNR_list.txt', 'w')
        for element in snr_list:
            file.write(str(element)+ '\n')
        file_name=path +'/coadded_SOPHIE '+ star_name +'.fits'
        ispec.write_spectrum(coadded_spectrum, file_name)
    else:
        file=open(path+'/Spectra_too_low_SNR', 'w')
        for element in snr_list:
            file.write(str(element)+ '\n')
      

The function below makes the coadding occur separately for the two different modes, as well as setting a wavelength range which can be changed if necessary.

In [4]:
def coadd_separate(star_name):
    path = p +star_name
    if os.path.exists(path + '/HR'):
        HR_raw_spectra = [i for i in os.listdir(path+'/HR') if i.endswith('.fits')]
        coadd_spectra(star_name=star_name, mode = 'HR', wavelength_min=420, wavelength_max=680)
    if os.path.exists(path+'/HE'):
        HE_raw_spectra = [i for i in os.listdir(path+'/HE') if i.endswith('.fits')]
        coadd_spectra(star_name=star_name, mode = 'HE', wavelength_min=420, wavelength_max=680)

parameters_from_ew generates a set of initial parameters that are used as an input for the synthesis method. The equivalent widths method is used to analyse only iron lines. Files are saved that contain the estimated parameters, and also an abundance plot.

In [5]:
def parameters_from_ew(star_name, mode='HR', code = 'moog', mask='G2'):
    if mode == 'HR':
        path = p +star_name +'/HR/'
        to_resolution=47000
    if mode == 'HE':
        path = p +star_name +'/HE/'
        to_resolution=40000
    #path = '/home/alix/Sophie data/SOPHIE_sd1_data/sd1_data/' +star_name+'/'
    file_name = 'coadded_SOPHIE '+ star_name+'.fits'
    spectrum= ispec.read_spectrum(path+file_name)
    #to_resolution=47000
    
    ####=============fitting line regions===========================
    
    if code in ['width', 'moog']:
            line_regions_with_atomic_data = ispec.read_line_regions(ispec_dir + "input/regions/47000_GES/{}_ew_ispec_good_for_params_all_extended.txt".format(code))
            
    else:
        line_regions_with_atomic_data = ispec.read_line_regions(ispec_dir + "/input/regions/47000_GES/{}_synth_good_for_params_all_extended.txt".format(code))
            
    # Select only iron lines
    line_regions_with_atomic_data = line_regions_with_atomic_data[np.logical_or(line_regions_with_atomic_data['note'] == "Fe 1", line_regions_with_atomic_data['note'] == "Fe 2")]

    smoothed_star_spectrum = ispec.convolve_spectrum(spectrum, 2*to_resolution)
    line_regions_with_atomic_data = ispec.adjust_linemasks(smoothed_star_spectrum, line_regions_with_atomic_data, max_margin=0.5)
    star_continuum_model = ispec.fit_continuum(spectrum, fixed_value=1.0, model="Fixed value")
    #--- Fit the lines but do NOT cross-match with any atomic linelist since they already have that information
    linemasks = ispec.fit_lines(line_regions_with_atomic_data, spectrum , star_continuum_model,\
                                    atomic_linelist = None, \
                                    max_atomic_wave_diff = 0.005,\
                                    check_derivatives = False, \
                                    discard_gaussian=False, \
                                    smoothed_spectrum=None, \
                                    discard_voigt=True, \
                                    free_mu=True, crossmatch_with_mu=False, closest_match=False)
    # Discard lines that are not cross matched with the same original element stored in the note
    #linemasks = linemasks[linemasks['element'] == line_regions['note']]

    # Exclude lines that have not been successfully cross matched with the atomic data
    # because we cannot calculate the chemical abundance (it will crash the corresponding routines)
    rejected_by_atomic_line_not_found = (linemasks['wave_nm'] == 0)
    linemasks = linemasks[~rejected_by_atomic_line_not_found]
    
    ###=============discarding any bad masks===========================
        
    flux_peak = spectrum['flux'][linemasks['peak']]
    flux_base = spectrum['flux'][linemasks['base']]
    flux_top = spectrum['flux'][linemasks['top']]
    bad_mask = np.logical_or(linemasks['wave_peak'] <= linemasks['wave_base'], linemasks['wave_peak'] >= linemasks['wave_top'])
    bad_mask = np.logical_or(bad_mask, flux_peak >= flux_base)
    bad_mask = np.logical_or(bad_mask, flux_peak >= flux_top)
    linemasks = linemasks[~bad_mask]
    
    #================Exclude lines with EW equal to zero=========
    
    rejected_by_zero_ew = (linemasks['ew'] == 0)
    linemasks = linemasks[~rejected_by_zero_ew]

    #================Exclude lines that may be affected by tellurics========
    
    rejected_by_telluric_line = (linemasks['telluric_wave_peak'] != 0)
    linemasks = linemasks[~rejected_by_telluric_line]
    
    #================Model spectra from EW===========================
    if mask =='G2':
        initial_teff = 6000
        initial_logg = 4.00
        initial_MH = 0.00
        initial_alpha = 0.00
        initial_vmic = ispec.estimate_vmic(initial_teff, initial_logg, initial_MH)
        
    if mask == 'K5':
        initial_teff=4440
        initial_logg=4.3
        initial_MH = 0.00
        initial_alpha = 0.00
        initial_vmic = ispec.estimate_vmic(initial_teff, initial_logg, initial_MH)
    max_iterations = 10
    
    model = ispec_dir + "/input/atmospheres/MARCS.GES/"
    atomic_linelist_file = ispec_dir + "/input/linelists/transitions/GESv6_atom_hfs_iso.420_920nm/atomic_lines.tsv"  
    
    
    solar_abundances_file = ispec_dir + "/input/abundances/Grevesse.2007/stdatom.dat"
    # Load model atmospheres
    modeled_layers_pack = ispec.load_modeled_layers_pack(model)

    # Load SPECTRUM abundances
    solar_abundances = ispec.read_solar_abundances(solar_abundances_file)


    # Validate parameters
    if not ispec.valid_atmosphere_target(modeled_layers_pack, {'teff':initial_teff, 'logg':initial_logg, 'MH':initial_MH, 'alpha':initial_alpha}):
        msg = "The specified effective temperature, gravity (log g) and metallicity [M/H] \
                fall out of theatmospheric models."
        print(msg)
    
    # Reduced equivalent width
    # Filter too weak/strong lines
    # * Criteria presented in paper of GALA
    
    efilter = np.logical_and(linemasks['ewr'] >= -6.0, linemasks['ewr'] <= -4.3)
    # Filter high excitation potential lines
    # * Criteria from Eric J. Bubar "Equivalent Width Abundance Analysis In Moog"
    efilter = np.logical_and(efilter, linemasks['lower_state_eV'] <= 5.0)
    efilter = np.logical_and(efilter, linemasks['lower_state_eV'] >= 0.5)
    ## Filter also bad fits
    efilter = np.logical_and(efilter, linemasks['rms'] < 1.00)
    # no flux
    noflux = spectrum['flux'][linemasks['peak']] < 1.0e-10
    efilter = np.logical_and(efilter, np.logical_not(noflux))
    unfitted = linemasks['fwhm'] == 0
    efilter = np.logical_and(efilter, np.logical_not(unfitted))

    results = ispec.model_spectrum_from_ew(linemasks[efilter], modeled_layers_pack, \
                        solar_abundances, initial_teff, initial_logg, initial_MH, initial_alpha, initial_vmic, \
                        free_params=["teff", "logg"], \
                        adjust_model_metalicity=True, \
                        max_iterations=max_iterations, \
                        enhance_abundances=True, \
                        #outliers_detection = "robust", \
                        #outliers_weight_limit = 0.90, \
                        outliers_detection = "sigma_clipping", \
                        #sigma_level = 3, \
                        tmp_dir = None, \
                        code=code)
    
    params, errors, status, x_over_h, selected_x_over_h, fitted_lines_params, used_linemasks = results
    data = []
    for key,value in params.items():
        data.append([key,value])
    errs=[]
    for key,value in errors.items():
        errs.append(value)
        
        
    
    df = pd.DataFrame(data, columns=['Parameter','Value'])
    df['Errors']=errs
    df.to_csv(path +'params.csv', index=False)
    
    stat=[]
    for  key,value in status.items():
        stat.append([key,value])
    df=pd.DataFrame(stat)
    df.to_csv(path +'status.csv', index=False)
    
    df =pd.DataFrame(fitted_lines_params)
    df.to_csv(path +'fitted_line_params.csv', index=False)
    
    df =pd.DataFrame(used_linemasks)
    df.to_csv(path +'used_linemasks.csv', index=False)
    
    np.savetxt(path + '/x_over_h_ew.csv',x_over_h, delimiter=',')
    file=open(path +'/selected_x_over_h.txt', 'w')
    for element in selected_x_over_h:
        file.write(str(element)+ '\n')
        
    lower_state_e=[row[4] for row in used_linemasks]
    element = [row[0] for row in used_linemasks]
    lower_state_ev_1 = [lower_state_e[i] for i in range(len(lower_state_e)) if element[i]=='Fe 1' ] 
    lower_state_ev_2 = [lower_state_e[i] for i in range(len(lower_state_e)) if element[i]=='Fe 2' ] 
    
    x_h = [i for i in x_over_h if str(i) != 'nan']
    avg = [mean(x_h) for i in range(len(lower_state_e))]
    x_h_1 = [x_h[i] for i in range(len(x_h)) if element[i]=='Fe 1' ] 
    x_h_2 = [x_h[i] for i in range(len(x_h)) if element[i]=='Fe 2' ] 
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.scatter(lower_state_ev_1, x_h_1)
    ax.scatter(lower_state_ev_2, x_h_2, c='orange')
    ax.plot(lower_state_e, avg, color='red')
    ax.legend(['[M/H]','Fe 1', 'Fe 2'])
    plt.ylim([-1.0, 1.0])
    plt.xlabel('Lower State (eV)')
    plt.ylabel('[M/H]')
    plt.savefig(path+'/EW_abundances.jpg', bbox_inches='tight', dpi=100)
    plt.show()
    
    return params

Cell below ensures that equivalent widths analysis is done for the modes separately.

In [6]:
def params_ew_separate(star_name, mask='G2'):
    path = p +star_name
    if os.path.exists(path + '/HR/coadded_SOPHIE '+ star_name+'.fits'):
        
        HR_ew = parameters_from_ew(star_name=star_name, mode = 'HR')
    else:
        HR_ew = 0
    if os.path.exists(path+'/HE/coadded_SOPHIE '+ star_name+'.fits'):
        
        HE_ew = parameters_from_ew(star_name=star_name, mode = 'HE')
    else:
        HE_ew = 0
    return HR_ew, HE_ew

The bulk of the code is below - this does the synthesis method on the spectra. This can take quite a long time; on average for me this was about 1 to 2 hours. Files are saved that contain the derived parameters, and a fits file that contains the synthesised spectrum. If comparing the synthesised spectrum to the observed (coadded) one, keep in mind that there are a lot of lines that the synthesis method won't fit, so if many appear to be missing this isn't a problem. Synthesis analysis is only done based on lines that actually fit to the data with a decent enough chi-squared. 

The free parameters can be changed in this function - I've made a comment where you can change them, and a full list of the parameters that can be set as free. If you have a reliable estimate for a specific parameter, change its value below the Model spectra line, and remove it from the free parameter list further down.

In [8]:
def merged_synthesis(star_name, ew_res, code='spectrum', mode = 'HR'):
    if mode == 'HR':
        path = p +star_name +'/HR/'
        atomic_linelist_file = ispec_dir + '/input/linelists/transitions/sousa_lines.tsv' 
    if mode == 'HE':
        path = p +star_name +'/HE/'
        atomic_linelist_file = ispec_dir + '/input/linelists/transitions/atomic_lines_bad_removed.tsv'
        to_resolution= 40000
    
    spectrum= ispec.read_spectrum(path+file_name)
    
    #spectrum = ispec.read_spectrum(merged_spectrum)
    print('read spectrum')
    

    print('assigned res')
    #===========Model spectra===========================
    # Parameters
    initial_teff =ew_res['teff']
    initial_logg =ew_res['logg']
    initial_MH = ew_res['MH']
    initial_alpha = ew_res['alpha']#ispec.determine_abundance_enchancements(initial_MH) #use input from ew
    initial_vmic = ew_res['vmic']
    #ispec.estimate_vmic(initial_teff, initial_logg, initial_MH)
    initial_vmac = ispec.estimate_vmac(initial_teff, initial_logg, initial_MH)
    #ispec.estimate_vmac(initial_teff, initial_logg, initial_MH)
    initial_vsini = 1.27332434061464
    initial_limb_darkening_coeff = 0.6
    initial_R = to_resolution
    initial_vrad = 0
    max_iterations = 6 #CHANGE THIS
    print('set params')
    # Selected model amtosphere, linelist and solar abundances
    
    model = ispec_dir + "/input/atmospheres/MARCS.GES/"
    print('model selected')
    
    #GESv6_atom_hfs_iso.420_920nm/atomic_lines.tsv"
   
    print('linelist selected')
    if "ATLAS" in model:
        solar_abundances_file = ispec_dir + "/input/abundances/Grevesse.1998/stdatom.dat"
    else:
        # MARCS
        solar_abundances_file = ispec_dir + "/input/abundances/Grevesse.2007/stdatom.dat"
    #solar_abundances_file = ispec_dir + "/input/abundances/Asplund.2005/stdatom.dat"
    #solar_abundances_file = ispec_dir + "/input/abundances/Asplund.2009/stdatom.dat"
    #solar_abundances_file = ispec_dir + "/input/abundances/Anders.1989/stdatom.dat"
    print('solar abundances loaded')
    isotope_file = ispec_dir + "/input/isotopes/SPECTRUM.lst"

    # Load chemical information and linelist
    atomic_linelist = ispec.read_atomic_linelist(atomic_linelist_file, wave_base=np.min(spectrum['waveobs']), wave_top=np.max(spectrum['waveobs']))
    atomic_linelist = atomic_linelist[atomic_linelist['theoretical_depth'] >= 0.01] # Select lines that have some minimal contribution in the sun

    isotopes = ispec.read_isotope_data(isotope_file)
    print('linelist and isotope files read')
    # Load model atmospheres
    modeled_layers_pack = ispec.load_modeled_layers_pack(model)

    # Load SPECTRUM abundances
    solar_abundances = ispec.read_solar_abundances(solar_abundances_file)
    print('model and abundances loaded')
    
    # Free parameters - these can be changed
    #free_params = ["teff", "logg", "MH", "vmic", "vmac", "vsini", "R", "vrad", "limb_darkening_coeff"]
    
    
    
    free_params = ["teff","logg","mh", "vmac", "vsini", "limb_darkening_coeff"]
    print('free parameters set')
    # Free individual element abundance
    
    chemical_elements_file = ispec_dir + "/input/abundances/chemical_elements_symbols.dat"
    chemical_elements = ispec.read_chemical_elements(chemical_elements_file)
    #free_abundances = ispec.create_free_abundances_structure(["He"], chemical_elements, solar_abundances)
    #free_abundances['Abund'] += initial_MH # Scale to metallicity
    free_abundances = None
    linelist_free_loggf = None

    # Line regions
    line_regions = ispec.read_line_regions(ispec_dir + "/input/regions/47000_GES/{}_synth_good_for_params_all_extended.txt".format(code))
    print('line regions read')
    ## Select only some lines to speed up the execution if desired, but this isn't recommended
   # line_regions = line_regions[np.logical_or(line_regions['note'] == 'Ti 1', line_regions['note'] == 'Ti 2')]
    #line_regions = ispec.adjust_linemasks(spectrum, line_regions, max_margin=0.5)
    #line_regions = line_regions[np.logical_or(line_regions['note'] == 'Fe 1', line_regions['note'] == 'Fe 2')]
    line_regions = ispec.adjust_linemasks(spectrum, line_regions, max_margin=0.5)
    # Read segments if we have them or...
    #segments = ispec.read_segment_regions(ispec_dir + "/input/regions/fe_lines_segments.txt")
    # ... or create the segments
    segments = ispec.create_segments_around_lines(line_regions, margin=0.25)
    
    print('segments created')
    
    star_continuum_model = ispec.fit_continuum(spectrum, fixed_value=1.0, model="Fixed value")
    print('continuum fit')
    
    obs_spec, modeled_synth_spectrum, params, errors, abundances_found, loggf_found, status, stats_linemasks = \
            ispec.model_spectrum(spectrum, star_continuum_model, \
            modeled_layers_pack, atomic_linelist, isotopes, solar_abundances, free_abundances, linelist_free_loggf, initial_teff, \
            initial_logg, initial_MH, initial_alpha, initial_vmic, initial_vmac, initial_vsini, \
            initial_limb_darkening_coeff, initial_R, initial_vrad, free_params, segments=segments, \
            linemasks=line_regions, \
            enhance_abundances=True, \
            use_errors = True, \
            vmic_from_empirical_relation = False, \
            vmac_from_empirical_relation = True, \
            max_iterations=max_iterations, \
            tmp_dir = None, \
                                 
            code=code)
    
    #chisquared of how each line fits 
    
    data = []
    for key,value in params.items():
        data.append([key,value])
    errs=[]
    for key,value in errors.items():
        errs.append(value*3)
        
        
    
    df = pd.DataFrame(data, columns=['Parameter','Value'])
    df['Errors']=errs
    mh = df.at[2, 'Value']
    alpha = df.at[3, 'Value']
    feh = mh - math.log((0.694*(10**alpha))+0.306)
    feh_err = df.at[2, 'Errors']
    fe_dict = {'Parameter':'Fe/h', 'Value' : feh, 'Errors':feh_err}
    print(fe_dict)
    df=df.append(fe_dict, ignore_index=True)
    df.to_csv(path +'params_synth_pipeline.csv', index=False)
    
    stat=[]
    for  key,value in status.items():
        stat.append([key,value])
    df=pd.DataFrame(stat)
    df.to_csv(path +'status_synth_pipeline.csv', index=False)
    
    df =pd.DataFrame(stats_linemasks)
    df.to_csv(path +'stats_linemasks_synth_pipeline.csv', index=False)
    
    
    wavelengths = np.arange(420, 680, 0.001)
    resampled_star_spectrum = ispec.resample_spectrum(modeled_synth_spectrum, wavelengths, method='linear', zero_edges=True)
    file_name=path +'/synthesised_spectrum_pipeline'+ star_name +'.fits'
    ispec.write_spectrum(resampled_star_spectrum, file_name)
    
    print(params, errors, status)

Functions are combined into a pipeline below. The line mask affects the seed parameters - this is set to 'G5' as a default, however a 'K5' mask is also available as an input.

In [9]:
def pipeline(star_name, line_mask):
    separate_modes(star_name=star_name)
    coadd_separate(star_name=star_name)
    ew_results = params_ew_separate(star_name=star_name, mask=line_mask)
    if ew_results[0] != 0 :
        merged_synthesis(star_name=star_name, ew_res=ew_results[0], mode='HR')
    if ew_results[1] != 0:
        merged_synthesis(star_name=star_name, ew_res=ew_results[1], mode='HE')

## 